In [143]:
import json
import urllib
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import KFold
import hashlib
import numpy as np
import csv
from datetime import datetime, timedelta
from collections import Counter
import matplotlib.pyplot as plt
#%matplotlib inline
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import pickle
from sklearn import neighbors
from zipfile import ZipFile
from sklearn import svm
from pylab import *
import requests
import configparser
import tweepy
from TwitterAPI import TwitterAPI
import sys
from io import BytesIO
from zipfile import ZipFile
#from urllib.request import urlopen
from collections import Counter, defaultdict

In [144]:
def read_tweets(filename):
    tweet_t = []
    user_n = []
    with open(filename,) as csvfile:
        filereader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in filereader:
            try:
                tweet_t.append(row[2])
                user_n.append(row[3])
            except IndexError:
                pass
    return tweet_t,user_n

tweet_test, user_name = read_tweets('C:\\Users\\manas\\Desktop\\twitter_data.csv')

def tokenize(text):
    tokens = re.findall(r"\w+|\S", text.lower(),flags = re.L)
    tokens1 = []
    for i in tokens:
        x = re.findall(r"\w+|\S", i,flags = re.U)
        for j in x:
            tokens1.append(j)
            
    return tokens1

tokens = [tokenize(t) for t in tweet_test]

In [145]:
afinn_file = open('AFINN-111.txt', 'r')

afinn = dict()

for line in afinn_file:
    parts = line.strip().split()
    if len(parts) == 2:
        afinn[parts[0].decode("utf-8")] = int(parts[1])

def afinn_sentiment2(terms, afinn, verbose=False):
    pos = 0
    neg = 0
    for t in terms:
        if t in afinn:
            if verbose:
                print('\t%s=%d' % (t, afinn[t]))
            if afinn[t] > 0:
                pos += afinn[t]
            else:
                neg += -1 * afinn[t]
    return pos, neg

#manually labelling the data
positives = []
negatives = []
all2 = [] 
all1 = []
neutral = []
tweet_manual_labelling = []

for token_list, tweet in zip(tokens, tweet_test):
    pos, neg = afinn_sentiment2(token_list, afinn)
    all2.append((tweet, pos, neg))
    if pos > neg:
        positives.append((tweet, pos, neg))
        #print(positives)
    elif neg > pos:
        negatives.append((tweet, pos, neg))
    else:
        all1.append((tweet, pos, neg))
        
for tweet, pos, neg in sorted(positives, key=lambda x: x[1], reverse=True):
    neutral.append(('2', tweet))


for tweet, pos, neg in sorted(negatives, key=lambda x: x[1], reverse=True):
    neutral.append(('1', tweet))

for tweet, pos, neg in sorted(all1, key=lambda x: x[1], reverse=True):
    neutral.append(('0', tweet))
                    
with open('neutral.csv', 'w') as fp1:
    filewriter = csv.writer(fp1)
    filewriter.writerows(neutral)
    
print(len(positives),len(negatives),len(all1))

(124, 24, 152)


In [86]:
'''tweet_manual_labelling = []
for i in tweet_test:
    #if i.split()[0] != 'rt':
    tweet_manual_labelling.append((1,i))
with open('tweet_manual_labelling.csv', 'w') as fp1:
    filewriter = csv.writer(fp1)
    filewriter.writerows(tweet_manual_labelling)

tweet_test, user_name = read_tweets('twitter_data.csv')'''

"tweet_manual_labelling = []\nfor i in tweet_test:\n    #if i.split()[0] != 'rt':\n    tweet_manual_labelling.append((1,i))\nwith open('tweet_manual_labelling.csv', 'w') as fp1:\n    filewriter = csv.writer(fp1)\n    filewriter.writerows(tweet_manual_labelling)\n\ntweet_test, user_name = read_tweets('twitter_data.csv')"

In [146]:
def read_tarining_data(filename):
    labeled_tweets = []
    labels1 = []
    with open(filename, 'rb') as csvfile1:
        filereader = csv.reader(csvfile1)
        for  row in filereader:
            labeled_tweets.append(row[1])
            labels1.append(int(row[0]))
    return labeled_tweets,np.array(labels1)

labeled_tweets,labels = read_tarining_data('neutral.csv')

In [147]:
def do_vectorize(tokenizer_fn=tokenize, min_df=1,
                 max_df=1., binary=True, ngram_range=(1,1)):

    lis = ['weed','marijuana','pot','cannabis']#list of stop words
    #countvectorizer object
    vectorizer = CountVectorizer(input = 'content', tokenizer = tokenizer_fn, min_df=min_df, 
                                     max_df=max_df, binary=binary, ngram_range=ngram_range,
                                 dtype = 'int',analyzer='word',token_pattern='(?u)\b\w\w+\b',encoding='utf-8' )
    return vectorizer

In [148]:
vectorizer = do_vectorize() #get the vectorizer object
matrix = vectorizer.fit_transform(labeled_tweets) # learn new words and tranform it to a CSR matrix
#print(matrix.toarray())
print ('matrix represents %d documents with %d features' % (matrix.shape[0], matrix.shape[1]))
print('first doc has terms:\n%s' % (str(sorted(matrix[0].nonzero()[1]))))

matrix represents 300 documents with 923 features
first doc has terms:
[27, 93, 151, 209, 226, 501, 570, 620, 773, 797, 798, 811, 845, 862, 895]


In [149]:
def repeatable_random(seed):
    hash = str(seed)
    while True:
        hash = hashlib.md5(hash).digest()
        #print(type(hash))
        for c in hash:
            #print(type(c))
            yield ord(c)

def repeatable_shuffle(X, y, labeled_tweets):
    r = repeatable_random(42) 
    indices = sorted(range(X.shape[0]), key=lambda x: next(r))
    #print(indices)
    return X[indices], y[indices], np.array(labeled_tweets)[indices]

X, y, twittertweets = repeatable_shuffle(matrix, labels, labeled_tweets)

print('fourth shuffled document %s has label %d and terms: %s' % 
      (twittertweets[4], y[4], sorted(X[4].nonzero()[1])))

fourth shuffled document make memories with maui motorhomes by grabbing this incredible deal this blackfriday for bookings click this_is_a_url has label 0 and terms: [112, 118, 131, 165, 224, 317, 362, 413, 486, 490, 498, 511, 797, 798, 898]


In [150]:
def accuracy(truth, predicted):
    return len(np.where(truth==predicted)[0]) / len(truth)

predicted = model.predict(X)
print('accuracy on training data=%.3f' % accuracy(y, predicted))

accuracy on training data=1.000


In [151]:
def get_clf(C = 1.,penalty = 'l2', sample_weight=None):
    return LogisticRegression(C = C,penalty=penalty,random_state=42)
clf_logistic = get_clf()

def get_clf_SVC(C = 1.,penalty = 'l2', sample_weight=None):
    return LinearSVC(C = C,penalty=penalty,random_state=42)
clf_lSVC = get_clf_SVC()



def do_cross_validation(X, y,clf, n_folds):
    cv = KFold(len(y), n_folds)
    accuracies = []
    for train_ind, test_ind in cv: 
        #print(y[train_ind])
        #print('\n')
        #y[train_ind].to_csv(path='ytard.csv')
        #Z = np.c_[X.reshape(len(X),-1),y.reshape(len(y),-1)]
        #new_X = Z[:,:X.size//len(X)].reshape(X.shape)
        #new_y=Z[:,X.size//len(X):].reshape(y.shape)
        clf.fit(X[train_ind], y[train_ind])
        predictions = clf.predict(X[test_ind])
        #print(predictions)
        accuracies.append(accuracy_score(y[test_ind], predictions))
    avg = np.mean(accuracies)
    return avg

In [152]:
logistic_regression_accuracy = (do_cross_validation(X, y,clf_logistic, 5))*100
print('Average cross validation accuracy for Logistic Regression=%.1f percentage' % (logistic_regression_accuracy))

linear_regression_accuracy=(do_cross_validation(X, y,clf_lSVC, 5)*100)
print('Average cross validation accuracy for LinearSVC=%.1f percentage' % (linear_regression_accuracy))

Average cross validation accuracy for Logistic Regression=81.3 percentage
Average cross validation accuracy for LinearSVC=82.3 percentage


In [153]:
C_list = [0.1,1.0,10.,50.,100.]
lr_acr = []
ls_acr = []
for i in C_list:
    clf_logistic1 = get_clf(C=i)
    clf_lSVC1 = get_clf_SVC(C=i)
    lr_acr.append(do_cross_validation(X, y,clf_logistic1,5)*100)
    ls_acr.append(do_cross_validation(X, y,clf_lSVC,5)*100)
print "C\tLogistic_Regression\tLinearSVC"
for i in range(len(C_list)):
    print "%.1f\t%.1f\t\t\t%.1f" %(C_list[i],lr_acr[i],ls_acr[i])

C	Logistic_Regression	LinearSVC
0.1	78.0			82.3
1.0	81.3			82.3
10.0	82.0			82.3
50.0	82.3			82.3
100.0	82.0			82.3


In [154]:
def prediction(CLF,trained_CSR,trained_label,untrained_tweets_CSR):
    CLF.fit(trained_CSR,trained_label)
    predicted = CLF.predict(untrained_tweets_CSR)
    return predicted

In [166]:
def read_tweets(filename):
    tweet_t = []
    user_n = []
    with open(filename, 'rb') as csvfile:
        filereader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in filereader:
            try:
                tweet_t.append(row[2])
                user_n.append(row[3])
            except IndexError:
                pass
    return tweet_t,user_n

tweet_test1, user_name = read_tweets('twitter_data.csv')



In [167]:
X = vectorizer.fit_transform(x for x in twittertweets)
test_tweet_vector = vectorizer.transform(t for t in tweet_test1)

In [168]:
logitic_pred_dict = dict(Counter(Tweet_predicted_logistic))
linearsvc_pred_dict = dict(Counter(Tweet_predicted_linearSVC))
def print_results(dictionary):
    for i in dictionary:
        if i == 1:
            print "\tTweets aganist marijuana\t\t%d" %dictionary[i]
        elif i == 0:
            print "\tPro marijuana tweets\t\t\t%d" %dictionary[i]
        elif i == 2:
            print "\tTweets supporting its Medical use\t%d" %dictionary[i]

In [169]:
print "Logistic Regression Results"
print_results(logitic_pred_dict)

Logistic Regression Results
	Pro marijuana tweets			73
	Tweets aganist marijuana		7
	Tweets supporting its Medical use	20


In [170]:
print "SVC Results"
print_results(linearsvc_pred_dict)

SVC Results
	Pro marijuana tweets			68
	Tweets aganist marijuana		7
	Tweets supporting its Medical use	25
